In [ ]:
import regex as re
from tqdm import tqdm
import numpy as np

In [ ]:
testdata = """
10 players; last marble is worth 1618 points: high score is 8317
13 players; last marble is worth 7999 points: high score is 146373
17 players; last marble is worth 1104 points: high score is 2764
21 players; last marble is worth 6111 points: high score is 54718
30 players; last marble is worth 5807 points: high score is 37305
""".strip().splitlines()

In [ ]:
# We're now going to start storing the data in a file to avoid having to paste into the main document
# Make sure you create a file with the name below and save the real problem output there. 
# If you want to run just the sample data, skip this block
with open("./09-input.txt", "r") as FILE:
    data = FILE.read().strip().splitlines()

In [ ]:
pattern_input = re.compile("(\d+) players; last marble is worth (\d+) points(?:\: high score is (\d+))?")
all_data = testdata + data

In [ ]:
all_data

# Develop the game rules

Here's a simple implementation of the game rules just to verify that we can do the correct placement of the marbles etc

In [ ]:
# We prime the initial Round
current_marble = 0
board = [0]

for round in range(1,28):
    current_marble_pos = board.index(current_marble)
    current_marble = round

    if current_marble % 23 == 0:
        print("keeping", current_marble)
        new_pos = current_marble_pos - 7
        while new_pos<0:
            new_pos += len(board) 

        keep = board.pop(new_pos)
        print("  and keeping", keep)
        current_marble = board[new_pos]
    else:    
        new_pos = current_marble_pos + 2
        while new_pos>len(board):
            new_pos -= len(board) 
        board.insert(new_pos, round)
    
    output = "".join(["{0:2d}  ".format(r) for r in board])
    output = output.replace(" {0:2d} ".format(current_marble), "\033[1m({0:2d})\033[0m".format(current_marble))
    print("{0:2d}  ".format(round), output)
    


We now implement this as a function that takes number of players and number of marbles as input

In [ ]:
def play_game(players, marbles):
    # We prime the initial Round
    current_marble = 0
    current_marble_pos = 0
    board = [0]
    
    player_scores = dict()
    
    for round in tqdm(range(1, marbles+1)):
        current_marble_pos = board.index(current_marble)
        current_marble = round

        if current_marble % 23 == 0:
            current_player = round % players
            current_player_scores = player_scores.get(current_player, [])
            if len(current_player_scores) == 0:
                player_scores[current_player] = current_player_scores
                
            current_player_scores.append(current_marble)

            current_marble_pos = current_marble_pos - 7
            while current_marble_pos<0:
                current_marble_pos += len(board) 

            current_player_scores.append(board.pop(current_marble_pos))
            current_marble = board[current_marble_pos]
        else:    
            current_marble_pos = current_marble_pos + 2
            while current_marble_pos>len(board):
                current_marble_pos -= len(board) 
            board.insert(current_marble_pos, round)
            
    # Now calculate scores
    scores = {k: sum(v) for k,v in player_scores.items()}
            
    return player_scores, max(scores.values()), board


In [ ]:
for game in all_data:
    m = pattern_input.match(game)
    scores, score, board = play_game(int(m[1]), int(m[2]))
    print("{} players; last marble is worth {} points: high score is {}".format(m[1], m[2], score))
    if m[3] is not None:
        if int(m[3]) == score: 
            print(" ** CORRECT")

# Part 2

This one is easy, but might take some time to run based on the initial implementation. 

If we want to speed this up, why don't we implement the Marble's as a [doubly-linked list](https://en.wikipedia.org/wiki/Doubly_linked_list) instead? We shall create a Marble class 
that has a value and pointers to the clockwise and anticlockwise marbles. 

In [ ]:
class Marble(object):
    def __init__(self, value):
        """ Initialise as a single value-chain """
        self.__value = value
        self.__clockwise = self
        self.__anticlockwise = self
        
    def __str__(self):
        """ String representation simply the value """
        return str(self.__value)
        
    def value(self):
        """ Returns the value """
        return self.__value
        
    def insert(self, marble):
        """ Inserts a new value clockwise of itself """
        self.__clockwise.__anticlockwise = marble
        marble.__clockwise = self.__clockwise
        self.__clockwise = marble
        marble.__anticlockwise = self

    def delete(self):
        """ 
            Removes this marble. Marbles to either side will point at eachother instead, and
            this marble becomes a self-contained chain
        """
        
        self.__clockwise.__anticlockwise = self.__anticlockwise
        self.__anticlockwise.__clockwise = self.__clockwise
        
        self.__clockwise = self
        self.__anticlockwise = self
        
    def clockwise(self):
        """ Returns clockwise marble """
        return self.__clockwise

    def anticlockwise(self):
        """ Returns anticlockwise marble """
        return self.__anticlockwise


Create new game implementation using the Marble class

In [ ]:
def play_linked_game(players, marbles, debug=False):
    zero_marble = Marble(0)
    current_marble = zero_marble
    player_scores = dict()
    
    for round in tqdm(range(1, marbles+1)):
        if round % 23 == 0:
            current_player = round % players
            current_player_scores = player_scores.get(current_player, 0)
            current_player_scores += round
            
            for n in range(0,6):
                current_marble = current_marble.anticlockwise()
                
            delete_marble = current_marble.anticlockwise()
            current_player_scores += delete_marble.value()
            player_scores[current_player] = current_player_scores
            
            delete_marble.delete()

        else:    
            current_marble = current_marble.clockwise()
            current_marble.insert(Marble(round))
            current_marble = current_marble.clockwise()

    if debug:
        print_marble = zero_marble.clockwise()
        values = " 0"
        while print_marble != zero_marble:
            if print_marble == current_marble:
                values += " ({})".format(print_marble.value())
            else:
                values += " {}".format(print_marble.value())
            print_marble = print_marble.clockwise()
        print(values)
        
    # Now calculate scores
    return player_scores, max(player_scores.values())

play_linked_game(9, 25, debug=True)

In [ ]:
for game in all_data:
    m = pattern_input.match(game)
    scores, score = play_linked_game(int(m[1]), int(m[2]))
    print("{} players; last marble is worth {} points: high score is {}".format(m[1], m[2], score))
    if m[3] is not None:
        if int(m[3]) == score: 
            print(" ** CORRECT")

In [ ]:
for game in data:
    m = pattern_input.match(game)
    scores, score = play_linked_game(int(m[1]), 100 * int(m[2]))
    print("{} players; last marble is worth {} points: high score is {}".format(m[1], 100 * int(m[2]), score))